In [1]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
tt = time.time
st = tt()

In [2]:
# get prof name from thumbnail
def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def make_rgb_bgr(img):
    b,g,r = cv2.split(img)          
    rgb_img = cv2.merge([r,g,b]) 
    return rgb_img
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def get_prof_name(text):
    hon_list = ['prof','dr']
    ext_list = [' ', '.']
    hon_list = flatten_list([[w+ext for ext in ext_list] for w in hon_list])
    ishon=False
    text = text.split(' ')
    for hon in hon_list:
        if hon in text:
            idx = text.index(hon)
            if len(text[idx+2])<3:
                j = 4
            else:
                j = 3
            text = " ".join(text[idx:idx+j])
            return text
    return None
    
def get_name_fom_image(img):
    
    tesseract_file_path = "C:/Users/YASH/AppData/Local/Tesseract-OCR/tesseract.exe"
    pytesseract.pytesseract.tesseract_cmd = tesseract_file_path
#     img = subimg(img, u=0.7)

    # Convert the image to gray scale 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # Performing OTSU threshold 
    ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY) 

    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18)) 

    # Appplying dilation on the threshold image 
    dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 
    # dilation = thresh1

    # Finding contours 
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,  
                                                     cv2.CHAIN_APPROX_NONE) 

    # Creating a copy of image 
    im2 = img.copy() 
    text = None
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 

        # Cropping the text block for giving input to OCR 
        cropped = im2[y:y + h, x:x + w] 

        # Apply OCR on the cropped image 
        text_sub = pytesseract.image_to_string(cropped)
        text_sub = text_sub.lower().replace("\n"," ").strip()
        if text_sub=='':
            continue
        text_sub_name = get_prof_name(text_sub)
        if text_sub_name is not None:
            print(text_sub)
            text = text_sub_name
            break
    
    return text

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img


In [8]:
# save faces and names

base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'
known_faces_dir = os.path.join(base_dir, 'known_faces')
full_vedios_dir = os.path.join(base_dir, 'full_vedio')
paylist_list = [x[0] for x in os.walk(full_vedios_dir)][1:]

print(f"number of paylist: {len(paylist_list)}\n")
for playlist in paylist_list:
    gotname = False
    gotface = False
    playlistn = playlist.split('\\')[-1]
    print(f"\npaylist: {playlistn}")
    vedio_path_list = get_extension_files(playlist, ext='.mp4')
    print(f"number of vedios: {len(vedio_path_list)}")
    search_interval = 20
    vst = 5
    vet = vst + search_interval
    
    text_sub = get_prof_name(playlistn.lower())
    if text_sub is not None:
        print(f'name: {text_sub} in the title')
        prof_name = text_sub 
        gotname = True
        
    for vedio_path in vedio_path_list:
        if gotface:
            continue
        else:
            if gotname:
#                 vst+=search_interval
                vet+=search_interval
            else:
                pass
            

        vfname = vedio_path.split('\\')[-1]
        print(f"vedio: {vfname}")
        print(f"(vst, vet): {(vst, vet)}")
        video_capture = cv2.VideoCapture(vedio_path)
        fps = int(video_capture.get(cv2.CAP_PROP_FPS))
        st = tt()
        i=-1
        while video_capture.isOpened():
            i+=1
            if int(i/fps) < vst:
                ret, frame = video_capture.read()
                continue

            if int(i/fps) > vet:
                break
            
            ret, frame = video_capture.read()

            if i%fps==0:
#                 print(f"time(sec): {int(i/fps)}")
                if not gotname:
                    text = get_name_fom_image(frame)
                    if text is not None:
                        print(f'name: {text} at time: {int(i/fps)}')
                        prof_name = text
                        gotname = True
                    
                else:
                    if not is_valid_img(frame):
                        continue
                    face_locations = face_recognition.face_locations(frame)
                    if len(face_locations)==1:
                        face_tup = face_locations[0]
                        img = frame[face_tup[0]:face_tup[2], face_tup[3]:face_tup[1], :]
                        image_path = os.path.join(known_faces_dir, prof_name+'.jpg')
                        img = post_processing(img)
                        cv2.imwrite(image_path, img) 
                        gotface = True
                        print(f'face at time: {int(i/fps)}')
                        break

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()
        
    if gotface:
            continue
    else:
        if gotname:
            print(f"face not found for {prof_name}")
        else:
            print(f"face and name not found for {playlist}")

number of paylist: 10


paylist: Artificial Intelligence Search Methods for problem Solving
number of vedios: 5
vedio: AETGmGYy0UU.mp4
(vst, vet): (5, 25)
prof. deepak khemani department of computer science and engineering iit madras
name: prof. deepak khemani at time: 9
face at time: 16

paylist: Cloud Computing
number of vedios: 3
vedio: fZ3D6HQrWzs.mp4
(vst, vet): (5, 25)
vedio: NzZXz3fJf6o.mp4
(vst, vet): (5, 25)
vedio: SqG-b5E9vHs.mp4
(vst, vet): (5, 25)
face and name not found for C:\Users\YASH\nptel_face_recognition\full_vedio\Cloud Computing

paylist: Compiler Design
number of vedios: 5
vedio: 7oB0urhzndk.mp4
(vst, vet): (5, 25)
prof. s. k. aggarwal.
name: prof. s. k. aggarwal. at time: 13
face at time: 17

paylist: Computer Architecture Prof. Smruti Ranjan Sarangi
number of vedios: 5
name: prof. smruti ranjan in the title
vedio: 1rV15Z1mbwo.mp4
(vst, vet): (5, 45)
vedio: 5kpJDc664Vo.mp4
(vst, vet): (5, 65)
vedio: AnyJrRKHvhY.mp4
(vst, vet): (5, 85)
vedio: kNriKOJ-JuA.mp4
(vst,

In [5]:
# vedio_path = 'C:\\Users\\YASH\\nptel_face_recognition\\full_vedio\\Cloud Computing\\fZ3D6HQrWzs.mp4'
# video_capture = cv2.VideoCapture(vedio_path)
# fps = int(video_capture.get(cv2.CAP_PROP_FPS))
# st = tt()
# i=-1
# while video_capture.isOpened():
#     i+=1
#     if int(i/fps) < 5:
#         ret, frame = video_capture.read()
#         continue

#     if int(i/fps) > 20:
#         break

#     ret, frame = video_capture.read()

#     if i%fps==0:
# #                 print(f"time(sec): {int(i/fps)}")
#         if not gotname:
#             text = get_name_fom_image(frame)
#             if text is not None:
#                 print(f'name: {text} at time: {int(i/fps)}')
#                 prof_name = text
#                 gotname = True

#         else:
#             face_locations = face_recognition.face_locations(frame)
#             if len(face_locations)==1:
#                 face_tup = face_locations[0]
#                 img = frame[face_tup[0]:face_tup[2], face_tup[3]:face_tup[1], :]
#                 image_path = os.path.join(known_faces_dir, prof_name+'.jpg')
#                 cv2.imwrite(image_path, img) 
#                 gotface = True
#                 print(f'face at time: {int(i/fps)}')
#                 break

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# video_capture.release()
# cv2.destroyAllWindows()

face at time: 18


In [6]:
# plot_image_list(img_list[-2:])
get_name_fom_image(img_list[-1])

NameError: name 'img_list' is not defined